In [3]:
from csv import DictReader
import pandas as pd

In [ ]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        dict_reader = DictReader(f)
        
        # Read line by line 
        for row in dict_reader:
            

# Using Pandas

In [4]:
def pandas_read_file(file_path, chunksize=2**10):
    
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        process_chunk(chunk)
        
def process_chunk(chunk):
    print(chunk)

In [ ]:
pandas_read_file(, chunksize=2)

# Use dask or sframe